# Import

In [3]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import cv2
from subprocess import check_output
import os
import csv

# Parameters

In [4]:
trainImagePath='C:/Users/ANTHO/Desktop/TestTensorFlow/KAGGLE/Dataset/imgpp/'
labelsPath = 'C:/Users/ANTHO/Desktop/TestTensorFlow/KAGGLE/Dataset/GT/'

epochs=1
batchSize=1
learningRate=1e-4
dropOutFactor=0.5
trainSize=5
testSize=3


# Extract the images

In [5]:
#Extract all the file in the folder
#listImage=check_output(["ls", "Dataset/train"]).decode("utf8")
fileNames = os.listdir(trainImagePath)
fileNames = sorted(fileNames, key=lambda 
                    item: (int(item.partition('.')[0]) if item[0].isdigit() else float('inf'), item)) 

# select a subset of files
fileNames = fileNames[:]


#Print image
img = cv2.imread(trainImagePath+fileNames[0], cv2.IMREAD_COLOR)
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(img.shape)
#Size of the image (256,256, 3)

In [6]:
#Extract all the image flatten and write them in an array
imgs=[]
for fileName in fileNames:   
    img = cv2.imread(trainImagePath+fileName, cv2.IMREAD_COLOR)
    #cv2.IMREAD_COLOR
    #cv2.IMREAD_GRAYSCALE 
    img = img.ravel()#Flatten the image
    imgs.append(img)

imgs=np.matrix(imgs, dtype=float)/255
    
#Size(nbImage, nbPixel)
print(imgs.shape)

(10, 196608)


#Build back the image
imgBack=np.matrix(imgs[2])
imgBack = np.asarray(imgBack).reshape((256,256,3))
cv2.imshow('image',imgBack)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Extract the Labels

#Matrix with all the labels and 
with open(labelsPath, 'r') as f:
    reader = csv.reader(f)
    labels = np.matrix(list(reader), dtype=float)/100

In [7]:
fileNames = os.listdir(labelsPath)#Extract all the csv name of the folder
fileNames = sorted(fileNames, key=lambda 
                    item: (int(item.partition('GT.')[0]) if item[0].isdigit() else float('inf'), item))#Sort the files
labels=[]
for fileName in fileNames:
    with open(labelsPath+fileName, 'r') as f:#Read the file
        reader = csv.reader(f)
        label= np.asarray(list(reader), dtype=float)#Extract the value in an array
        label = label.ravel()#flatten
        labels.append(label)#Add to the list
labels = np.matrix(labels, dtype=float)#Convert to a matrix
print(labels.shape)

(10, 1849)


# Select size of train and test 

In [8]:
#Size of the training set
sizeTot= (imgs.shape[0]) #Nb of image in the dataset (from 0)

if (sizeTot-trainSize-testSize)<0:
    testSize=sizeTot-trainSize
    print('Not enough data for the test set')
    print('Only ', testSize, ' used for the test dataset')
#Validation set
imgsTest = imgs[(sizeTot-testSize):]
labelsHotTest = labels[(sizeTot-testSize):]


if trainSize>sizeTot:
    trainSize=sizeTot
    print('Size of the training data bigger than the dataset !')
#Training set
imgs=imgs[0:trainSize]
labelsHot=labels[0:trainSize]



# Convolutional Neural Network

In [9]:
#Function
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_label = tf.reshape(v_ys, [-1, 43, 43, 1])
    y_label = tf.cast(y_label, tf.float32)
    y_pre = sess.run(prediction, feed_dict={xs: v_xs})
    y_pre = tf.cast(y_pre, tf.float32)#Force float32
    #correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy = tf.reduce_sum(tf.square((tf.subtract(y_pre,  y_label))))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape=shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    # stride [1, x_movement, y_movement, 1]
    # Must have strides[0] = strides[3] = 1
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    # stride [1, x_movement, y_movement, 1]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def max_pool_3x3(x):
    # stride [1, x_movement, y_movement, 1]
    return tf.nn.max_pool(x, ksize=[1,3,3,1], strides=[1,3,3,1], padding='SAME')

In [11]:
## Architecture

## Input layer ##
with tf.name_scope("Input") as scope:
    # define placeholder for inputs to network
    xs = tf.placeholder(tf.float32, [None, imgs.shape[1]])   # 256x256x3
    ys = tf.placeholder(tf.float32, [None, labels.shape[1]]) #43x43 
    x_image = tf.reshape(xs, [-1, 256, 256, 3])#[batch, in_depth, in_height, in_width, in_channels].
    y_label = tf.reshape(ys, [-1, 43, 43, 1])
    # print(x_image.shape)  # [n_samples, 28,28,1]
    
## conv1 layer ##
## maxpooling 2x2 ##
#20x (7x7)
patch=7
sizeIn=3
sizeOut=20
with tf.name_scope("Conv1") as scope:
    W_conv1 = weight_variable([patch,patch, sizeIn,sizeOut])
    b_conv1 = bias_variable([sizeOut])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1) #Maxpooling 2x2, output size= inout size/2
    # Add summary ops to collect data
    w_h_conv1 = tf.summary.histogram("weightsConv1", W_conv1)
    b_h_conv1 = tf.summary.histogram("biasesConv1", b_conv1)
    
## conv2 layer ##
## maxpooling 3x3 ##
#40x (5x5)
patch=5
sizeIn=sizeOut #From previous ConvNet
sizeOut=40
with tf.name_scope("Conv2") as scope:
    W_conv2 = weight_variable([patch,patch, sizeIn,sizeOut])
    b_conv2 = bias_variable([sizeOut])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_3x3(h_conv2) #Maxpooling 3x3, output size= inout size/3
    # Add summary ops to collect data
    w_h_conv2 = tf.summary.histogram("weightsConv2", W_conv2)
    b_h_conv2 = tf.summary.histogram("biasesConv2", b_conv2)
    
## conv3 layer ##
#20x (5x5)
patch=5
sizeIn=sizeOut #From previous ConvNet
sizeOut=20
with tf.name_scope("Conv3") as scope:
    W_conv3 = weight_variable([patch,patch, sizeIn,sizeOut])
    b_conv3 = bias_variable([sizeOut])
    h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
    # Add summary ops to collect data
    w_h_conv3 = tf.summary.histogram("weightsConv3", W_conv3)
    b_h_conv3 = tf.summary.histogram("biasesConv3", b_conv3)
    
## conv4 layer ##
#10x (5x5)
patch=5
sizeIn=sizeOut #From previous ConvNet
sizeOut=10
with tf.name_scope("Conv4") as scope:
    W_conv4 = weight_variable([patch,patch, sizeIn,sizeOut])
    b_conv4 = bias_variable([sizeOut])
    h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4) + b_conv4)
    # Add summary ops to collect data
    w_h_conv4 = tf.summary.histogram("weightsConv4", W_conv4)
    b_h_conv4 = tf.summary.histogram("biasesConv4", b_conv4)
    
## conv5 layer ##
#1x (1x1)
patch=1
sizeIn=sizeOut #From previous ConvNet
sizeOut=1
with tf.name_scope("Conv5") as scope:
    W_conv5 = weight_variable([patch,patch, sizeIn,sizeOut])
    b_conv5 = bias_variable([sizeOut])
    h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5) + b_conv5)
    #h_conv5 = conv2d(h_conv4, W_conv5) + b_conv5
    # Add summary ops to collect data
    w_h_conv5 = tf.summary.histogram("weightsConv5", W_conv5)
    b_h_conv5 = tf.summary.histogram("biasesConv5", b_conv5)
    
   
 ## Prediction ##
with tf.name_scope("prediction") as scope:
    prediction = h_conv5
    img_prediction = tf.summary.image("densitymap", prediction)


## Loss function ##
# the error between prediction and real data
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),reduction_indices=[1]))# loss
with tf.name_scope("cost_function") as scope:
    cross_entropy = tf.reduce_sum(tf.square((tf.subtract(prediction, y_label))))#Euclidean distance
    tf.summary.scalar("cost_function", cross_entropy)
    
with tf.name_scope("train") as scope:
    #train_step = tf.train.AdamOptimizer(learningRate).minimize(cross_entropy)
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
    


In [12]:
#Training
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
# Merge all summaries into a single operator
merged_summary_op = tf.summary.merge_all()

with tf.Session() as sess:
#sess = tf.Session()
# important step
# tf.initialize_all_variables() no long valid from
# 2017-03-02 if using tensorflow >= 0.12

    sess.run(init)
    
    # Folder where the data are saved
    summary_writer = tf.summary.FileWriter('C:/Users/ANTHO/Desktop/TestTensorFlow/KAGGLE/Measure/Classes/', sess.graph)

    if int(len(imgs)) < batchSize:#If there are less data than the batch size
        nbBatch=1
    else:
        nbBatch=int((len(imgs))/batchSize)

    #print(compute_accuracy(imgsTest, labelsHotTest))
    for i in range(epochs):#Go through all the epochs
        for batchNum in range(nbBatch):#Go through all the batches
            if batchSize==1:
                batch_xs = imgs[batchNum*batchSize]
                batch_ys = labelsHot[batchNum*batchSize]
            else:
                batch_xs = imgs[batchNum*batchSize:(batchNum*batchSize+(batchSize-1))]
                batch_ys = labelsHot[batchNum*batchSize:(batchNum*batchSize+(batchSize-1))]
            
            sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys})
            print(compute_accuracy(imgsTest, labelsHotTest))
            
            summary_str = sess.run(merged_summary_op, feed_dict={xs: batch_xs, ys: batch_ys})
            summary_writer.add_summary(summary_str, i*int(len(imgs)/batchSize) + batchNum)
            #cost_history = np.append(cost_history,sess.run(cross_entropy,feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: dropOutFactor}))
            #weigthMeasure = sess.run(W_conv1,feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: dropOutFactor})
        #if i % 1 == 0:
            #print(i)
            #print(weigthMeasure)
            #print(compute_accuracy(imgsTest, labelsHotTest)) 



363.0
363.0
363.0
363.0
363.0
